In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
 
plt.style.use(['science','notebook'])
plt.style.reload_library()

In [3]:
from june import World 
from june.demography.geography import Geography
from june.demography import Demography
from june.interaction import Interaction
from june.infection import Infection, HealthIndexGenerator, InfectionSelector
from june.infection.transmission import TransmissionConstant
from june.groups import Hospitals, Schools, Companies, Households, CareHomes, Cemeteries, Universities
from june.groups.leisure import generate_leisure_for_config, Cinemas, Pubs, Groceries
from june.simulator import Simulator
from june.infection_seed import InfectionSeed
from june.policy import Policy, Policies
from june import paths
from june.hdf5_savers import load_geography_from_hdf5
from june.logger.read_logger import ReadLogger
from june.world import generate_world_from_geography
from june.hdf5_savers import generate_world_from_hdf5

No --data argument given - defaulting to:
/home/florpi/JUNE/data
No --configs argument given - defaulting to:
/home/florpi/JUNE/configs


# Initialize world

To initialize a certain world, we need to add the different components we want to have in it. First we specify what super areas (msoa) we want to create. We have included these ones, because they are known to contain hospitals, schools, care homes, and companies.

After creating the geography, we create the different components the worlds need to have such as care homes, companies ...

In [4]:
CONFIG_PATH = paths.configs_path / "config_example.yaml"

In [5]:
%%time 

geography = Geography.from_file(
        {
            "super_area": [
                "E02003282",
                "E02003283",
                "E02001720",
                "E02002560",
                "E02002559",
                "E02004314",
            ]
        }
)

geography.hospitals = Hospitals.for_geography(geography)
geography.schools = Schools.for_geography(geography)
geography.companies = Companies.for_geography(geography)
geography.care_homes = CareHomes.for_geography(geography)
geography.universities = Universities.for_super_areas(geography.super_areas)
world = generate_world_from_geography(geography, include_households=True, include_commute=True)


2020-08-30 17:51:08,806 - june.demography.geography - INFO - There are 162 areas and 6 super_areas in the world.
2020-08-30 17:51:08,811 - june.groups.hospital - INFO - There are 1 hospitals in this geography.
2020-08-30 17:51:08,880 - june.groups.school - INFO - There are 26 schools in this geography.
2020-08-30 17:51:08,942 - june.groups.school - INFO - No school for the age 0 in this world.
2020-08-30 17:51:08,946 - june.groups.school - INFO - No school for the age 1 in this world.
2020-08-30 17:51:08,949 - june.groups.school - INFO - No school for the age 2 in this world.
2020-08-30 17:51:09,206 - june.groups.carehome - INFO - There are 11 care_homes in this geography.
2020-08-30 17:51:19,364 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.
2020-08-30 17:51:23,706 - june.distributors.worker_distributor - INFO - There are 0 who had to be told to stay real
2020-08-30 17:51:43,802 - june.groups.commute.commutehub_distributor - INFO - Distributing people to commute hubs in 11 

## Commute, travel and leisure

In [6]:
world.pubs = Pubs.for_geography(geography)
world.cinemas = Cinemas.for_geography(geography)
world.groceries = Groceries.for_geography(geography)
leisure = generate_leisure_for_config(world, config_filename=CONFIG_PATH)
leisure.distribute_social_venues_to_households(world.households, super_areas=world.super_areas) # this assigns possible social venues to people.

2020-08-30 17:51:45,115 - june.groups.leisure.leisure - INFO - Distributing social venues to households
2020-08-30 17:51:45,115 - june.groups.leisure.leisure - INFO - Distributed in 0 of 20498 households.


We are also going to need some cemeteries...geography.cemeteries = Cemeteries()


In [7]:
world.cemeteries = Cemeteries()

In [8]:
len(world.people)

57259

### If it took a long time to run the previous commands, it might be a good idea to save the world to reuse it later.

In [9]:
world.to_hdf5("world.hdf5")

2020-08-30 17:51:53,659 - june.hdf5_savers.world_saver - INFO - saving world to HDF5
2020-08-30 17:51:54,549 - june.hdf5_savers.world_saver - INFO - saving hospitals...
2020-08-30 17:51:54,552 - june.hdf5_savers.world_saver - INFO - saving schools...
2020-08-30 17:51:54,558 - june.hdf5_savers.world_saver - INFO - saving companies...
2020-08-30 17:51:54,565 - june.hdf5_savers.world_saver - INFO - saving households...
2020-08-30 17:51:55,158 - june.hdf5_savers.world_saver - INFO - saving care homes...
2020-08-30 17:51:55,161 - june.hdf5_savers.world_saver - INFO - saving commute cities...
2020-08-30 17:51:55,166 - june.hdf5_savers.world_saver - INFO - saving commute hubs...
2020-08-30 17:51:55,171 - june.hdf5_savers.world_saver - INFO - saving universities...
2020-08-30 17:51:55,175 - june.hdf5_savers.world_saver - INFO - saving social venues...


If we would like to load the world we saved, we just do

In [10]:
world = generate_world_from_hdf5("world.hdf5")

2020-08-30 17:51:55,236 - june.hdf5_savers.world_saver - INFO - loading world from HDF5
2020-08-30 17:51:55,257 - june.hdf5_savers.world_saver - INFO - loading hospitals...
2020-08-30 17:51:55,262 - june.hdf5_savers.world_saver - INFO - loading schools...
2020-08-30 17:51:55,282 - june.hdf5_savers.world_saver - INFO - loading companies...
2020-08-30 17:51:55,316 - june.hdf5_savers.world_saver - INFO - loading care homes...
2020-08-30 17:51:55,323 - june.hdf5_savers.world_saver - INFO - loading universities...
2020-08-30 17:51:55,336 - june.hdf5_savers.world_saver - INFO - loading commute cities...
2020-08-30 17:51:55,381 - june.hdf5_savers.world_saver - INFO - loading commute hubs...
2020-08-30 17:51:55,501 - june.hdf5_savers.world_saver - INFO - loading households...
2020-08-30 17:51:55,840 - june.hdf5_savers.world_saver - INFO - loading population...
2020-08-30 17:51:56,208 - june.hdf5_savers.world_saver - INFO - loading social venues...
2020-08-30 17:51:56,215 - june.hdf5_savers.wor

In [11]:
# and regenerate leisure in case we load it externally
leisure = generate_leisure_for_config(world, CONFIG_PATH)

you have now a beautiful pre-pandemic world. 

# Adding the infection

The module in charge of infecting people is called the ``InfectionSelector``, which gives people a transmission time profile and a symptoms trajectory based on their age and sex (through the health index generator)

In [12]:
health_index_generator = HealthIndexGenerator.from_file(asymptomatic_ratio=0.2)
selector = InfectionSelector.from_file(
        health_index_generator=health_index_generator,
        transmission_config_path=paths.configs_path / 'defaults/transmission/XNExp.yaml'
)

# Adding the interaction

In [13]:
interaction = Interaction.from_file()

Beta are the intensities of the interaction taking place at the different groups

In [14]:
for key in interaction.beta.keys():
    interaction.beta[key] *= 0.2

In [15]:
interaction.beta

{'box': 0.2,
 'pub': 0.1,
 'grocery': 0.05,
 'cinema': 0.05,
 'commute_unit': 0.1,
 'commute_city_unit': 0.1,
 'hospital': 0.010000000000000002,
 'care_home': 0.24,
 'company': 0.16000000000000003,
 'school': 0.16000000000000003,
 'household': 0.2,
 'university': 0.1}

moreover this interaction module uses contact matrices, that are different for different groups. These contact matrices shouldnt be modified for now. However they are a combination of conversational contact matrices, and physical contact matrices (see the BBC pandemic paper, from where these matrices are extracted https://www.medrxiv.org/content/10.1101/2020.02.16.20023754v2)

There is a parameter, ``alpha`` ($\alpha$), that combines these two matrices in the following way,


$\beta M \left(1 + (\alpha -1) \right) P$

where $\beta$ is the intensity of the interaction, and $P$ the physical contact matrix. A larger $\alpha$ produces more physical contacts. It is an overall number, non dependent of the particular group.


In [16]:
interaction.alpha_physical

2.0

# Seed the disease

There are two options implemented in the seed at the moment, either you specify the number of cases and these are then homogeneously distributed by population to the different areas, or you use UK data on cases per region. For now use the first case.

In [17]:
infection_seed = InfectionSeed(
    world.super_areas, selector,
)

In [18]:
n_cases = 50
infection_seed.unleash_virus(n_cases) # play around with the initial number of cases

# Set policies

In [19]:
policies = Policies.from_file()

In [20]:
policies

# Run the simulation

The simulator is the main module in charge of running the simulation. It coordinates the ``ActivityManager`` which is responsible of allocating people to the right groups given the current timestep, it updates the health status of the population, and it runs the interaction over the different groups. All of these modules can be modified by policies at any given time.

Since the timer configuration is a bit cumbersome, it is read from the config file at ``configs/config_example.yaml``

In [21]:
simulator = Simulator.from_file(
    world=world,
    infection_selector=selector,
    interaction=interaction, 
    config_filename = CONFIG_PATH,
    leisure = leisure,
    policies = policies
)

In [22]:
%%time
simulator.run()

2020-08-30 17:52:04,083 - june.simulator - INFO - Starting group_dynamics for 40 days at day 0
2020-08-30 17:52:04,084 - june.simulator - INFO - starting the loop ..., at 0 days, to run for 40 days
2020-08-30 17:52:05,403 - june.simulator - INFO - Date = 2020-03-01 00:00:00, number of deaths =  0, number of infected = 50
2020-08-30 17:52:06,031 - june.simulator - INFO - Date = 2020-03-01 04:00:00, number of deaths =  0, number of infected = 50
2020-08-30 17:52:06,649 - june.simulator - INFO - Date = 2020-03-01 08:00:00, number of deaths =  0, number of infected = 50
2020-08-30 17:52:06,908 - june.simulator - INFO - Date = 2020-03-01 12:00:00, number of deaths =  0, number of infected = 50
2020-08-30 17:52:07,200 - june.simulator - INFO - Date = 2020-03-02 00:00:00, number of deaths =  0, number of infected = 50
2020-08-30 17:52:07,564 - june.simulator - INFO - Date = 2020-03-02 01:00:00, number of deaths =  0, number of infected = 50
2020-08-30 17:52:07,829 - june.simulator - INFO - Da

2020-08-30 17:52:33,686 - june.simulator - INFO - Date = 2020-03-14 08:00:00, number of deaths =  0, number of infected = 465
2020-08-30 17:52:33,960 - june.simulator - INFO - Date = 2020-03-14 12:00:00, number of deaths =  0, number of infected = 476
2020-08-30 17:52:34,570 - june.simulator - INFO - Date = 2020-03-15 00:00:00, number of deaths =  0, number of infected = 504
2020-08-30 17:52:35,192 - june.simulator - INFO - Date = 2020-03-15 04:00:00, number of deaths =  0, number of infected = 513
2020-08-30 17:52:35,839 - june.simulator - INFO - Date = 2020-03-15 08:00:00, number of deaths =  0, number of infected = 527
2020-08-30 17:52:36,112 - june.simulator - INFO - Date = 2020-03-15 12:00:00, number of deaths =  0, number of infected = 541
2020-08-30 17:52:36,612 - june.simulator - INFO - Date = 2020-03-16 00:00:00, number of deaths =  0, number of infected = 572
2020-08-30 17:52:37,240 - june.simulator - INFO - Date = 2020-03-16 01:00:00, number of deaths =  0, number of infecte

2020-08-30 17:53:20,590 - june.simulator - INFO - Date = 2020-03-28 04:00:00, number of deaths =  2, number of infected = 1769
2020-08-30 17:53:21,433 - june.simulator - INFO - Date = 2020-03-28 08:00:00, number of deaths =  2, number of infected = 1777
2020-08-30 17:53:22,062 - june.simulator - INFO - Date = 2020-03-28 12:00:00, number of deaths =  2, number of infected = 1792
2020-08-30 17:53:22,911 - june.simulator - INFO - Date = 2020-03-29 00:00:00, number of deaths =  2, number of infected = 1803
2020-08-30 17:53:23,750 - june.simulator - INFO - Date = 2020-03-29 04:00:00, number of deaths =  2, number of infected = 1802
2020-08-30 17:53:24,594 - june.simulator - INFO - Date = 2020-03-29 08:00:00, number of deaths =  2, number of infected = 1812
2020-08-30 17:53:25,225 - june.simulator - INFO - Date = 2020-03-29 12:00:00, number of deaths =  2, number of infected = 1816
2020-08-30 17:53:25,871 - june.simulator - INFO - Date = 2020-03-30 00:00:00, number of deaths =  2, number of 

While the simulation runs (and afterwards) we can launch the visualization webpage by running
```python june/visualizer.py path/to/results``` 

# Getting the results

All results are stored in a json file specified in the ``save_path`` variable in the config file. We can also access it from ``world.logger`` directly.

In [87]:
import pandas as pd

In [97]:
read = ReadLogger()

In [98]:
%%time
for i,area in enumerate(world.super_areas):
    read.process_symptoms(read.infections_per_super_area[i], n_people = 100000)

CPU times: user 663 ms, sys: 0 ns, total: 663 ms
Wall time: 666 ms


## Hospital data and how it changed over time

In [ ]:
hospitals_df = read.load_hospital_capacity()

In [ ]:
hospitals_df.head(3)

In [ ]:
hospitals_characteristics_df = read.load_hospital_characteristics()

In [ ]:
hospitals_characteristics_df

## where did infections happen?

In [ ]:
infection_locations = read.get_locations_infections()

In [ ]:
unique_locations, counts_locations = np.unique(
                np.array(infection_locations),
                return_counts=True)

location_counts_df = pd.DataFrame(
    {'locations': unique_locations,
    'counts': counts_locations}
)
location_counts_df.set_index('locations', inplace=True)

In [ ]:
location_counts_df['percent_infections']= 100*(location_counts_df['counts'])/location_counts_df.values.sum()

In [ ]:
import matplotlib.ticker as mtick

ax = location_counts_df['percent_infections'].sort_values().plot.bar()
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.ylabel('Percentage of infections at location')
plt.xlabel('location')


In [ ]:
# Find outbreaks

In [ ]:
unique_id, count_id =  np.unique(
        np.array(read.locations_df['location_id'].sum()),
        return_counts=True
)

In [ ]:
unique_id[np.argsort(count_id)[-5:]]

In [ ]:
count_id[np.argsort(count_id)[-5:]]

In [ ]:
weekly_infections=read.locations_df.resample('W').sum()

In [ ]:
for location in unique_locations:
    weekly_infections[str(location)] = weekly_infections.apply(
        lambda x: x.location.count(str(location)), axis=1
    )

In [ ]:
weekly_infections.iloc[:,3:].plot()

In [ ]:
for location in unique_locations:
    read.locations_df[str(location)] = read.locations_df.apply(
        lambda x: x.location.count(str(location)), axis=1
    )

In [ ]:
read.locations_df.index = pd.to_datetime(read.locations_df.index)


In [ ]:
read.locations_df.iloc[:,3:].plot()

## rate of infection

In [ ]:
r_df = read.get_r()

In [ ]:
from june.policy import SocialDistancing
r_df.plot()
colors = ["yellow", "orange", 'blue']
for policy in policies.policies:
    if isinstance(policy, SocialDistancing):
        plt.axvspan(policy.start_time, 
                policy.end_time,
                facecolor=colors.pop(),
                alpha=0.2)
plt.axhline(y=1, linestyle='dashed', color='gray')
plt.xlabel('Date')
plt.ylim(0,3)
plt.ylabel('R')

## World infection curves, and by super area

In [ ]:
%%time
world_df = read.world_summary()

In [ ]:
fig, ax = plt.subplots()
world_df['daily_infections'].resample('D').sum().plot(ax=ax)
for policy in policies.policies:
    plt.axvspan(policy.start_time, 
                policy.end_time,
                alpha=0.01)
plt.axvline('2020-03-23',
           linestyle='dashed',
           color='indianred')
ax.set_ylabel('# daily infections')
ax.set_xlabel('Date')

In [ ]:
fig, ax = plt.subplots()
world_df['daily_hospital_admissions'].resample('D').sum().plot(ax=ax)

plt.axvline('2020-04-01',
           linestyle='dashed',
           color='indianred')
ax.set_ylabel('# daily hospital admissions')
ax.set_xlabel('Date')

In [ ]:
fig, ax = plt.subplots()
world_df['current_infected'].plot(ax=ax)

ax.set_ylabel('# Infected')
ax.set_xlabel('Date')

In [ ]:
world_df.plot()

plt.legend(bbox_to_anchor=(1,1))


In [ ]:
read = ReadLogger()

In [ ]:
%%time

area_df = read.super_area_summary()

In [ ]:
area_df[area_df.super_area == "E02003282"].plot()
plt.legend(bbox_to_anchor=(1,1))

## World infection curves per age group

In [ ]:
ages_df = read.age_summary([0,10,20,30,40,
                  50,60,70,80,90,100])

In [ ]:
for name, group in ages_df.groupby('age_range'):
    group['current_infected'].plot(label=name)
colors = ["yellow", "orange","blue"]
for policy in policies.policies:
    if isinstance(policy, SocialDistancing):
        plt.axvspan(policy.start_time, 
                policy.end_time,
                facecolor=colors.pop(), alpha=0.2)
plt.legend(bbox_to_anchor=(1,1))


## Draw some of the symptoms trajectories

In [ ]:
random_trajectories = read.draw_symptom_trajectories(window_length=600,
                                        n_people=10)

In [ ]:
from june.infection import SymptomTag

In [ ]:
symptoms_values = [tag.value for tag in SymptomTag]
symptoms_names = [tag.name for tag in SymptomTag]


In [ ]:
for df_person in random_trajectories:
    df_person['symptoms'].plot()
plt.ylabel('Symptoms Trajectory')
_ = plt.yticks(symptoms_values, symptoms_names)
plt.xlabel('Date')

In [ ]:
for df_person in random_trajectories:
    df_person['n_secondary_infections'].plot()
plt.ylabel('Number of secondary infections')

plt.xlabel('Date')